In [ ]:
require 'torch'   -- torch
require 'image'   -- to visualize the dataset
require 'nn'      -- provides all sorts of trainable modules/layers
--require 'Dropout' -- Hinton dropout technique

if opt.type == 'cuda' then
   nn.SpatialConvolutionMM = nn.SpatialConvolution
end

----------------------------------------------------------------------
print(sys.COLORS.red .. '==> Define parameters')

-- 10-class problem: faces!
local noutputs = 10

-- Input dimensions
local nfeats = numberOfChannel
local width = imageWidth
local height = imageHeight

-- Hidden units, filter sizes (for ConvNet only)
local nstates = {16,32}
local filtsize = {5, 7}
local poolsize = 4

----------------------------------------------------------------------
print(sys.COLORS.red .. '==> Construct CNN')

local cnnNet = nn.Sequential()

-- stage 1: conv+max
cnnNet:add(nn.SpatialConvolutionMM(nfeats, nstates[1], filtsize[1], filtsize[1]))
cnnNet:add(nn.Threshold())
cnnNet:add(nn.SpatialMaxPooling(poolsize,poolsize,poolsize,poolsize))

-- stage 2: conv+max
cnnNet:add(nn.SpatialConvolutionMM(nstates[1], nstates[2], filtsize[2], filtsize[2]))
cnnNet:add(nn.Threshold())

local classifier = nn.Sequential()
-- stage 3: linear
classifier:add(nn.Reshape(nstates[2]))
classifier:add(nn.Linear(nstates[2], noutputs))

-- stage 4 : log probabilities
classifier:add(nn.LogSoftMax())

for _,layer in ipairs(cnnNet.modules) do
   if layer.bias then
      layer.bias:fill(.2)
      if i == #cnnNet.modules-1 then
         layer.bias:zero()
      end
   end
end

local model = nn.Sequential()
model:add(cnnNet)
model:add(classifier)

-- Loss: NLL
loss = nn.ClassNLLCriterion()

----------------------------------------------------------------------
print(sys.COLORS.red .. '==> Model:')
print(model)

if opt.type == 'cuda' then
   model:cuda()
   loss:cuda()
end

-- return package:
return {
   model = model,
   loss = loss
}